In [2]:
import pandas as pd
import numpy as np

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# https://baeseongsu.github.io/posts/mimiciii/
# 기초 테이블 확인 : https://mimic.mit.edu/docs/iii/tables/
# 스키마 확인 : https://mit-lcp.github.io/mimic-schema-spy/

# lab, pre, pro

In [3]:
import datetime as dt
def date_only(df,x):
    df[x] = pd.to_datetime(df[x])
    df[x] = df[x].dt.date

## lab

In [4]:
# lab
lab = pd.read_csv('폐렴환자lab.csv')
lab2 = lab.copy()
lab2 = lab2[['SUBJECT_ID','ITEMID','CHARTTIME','FLAG']]
lab2 = lab2[lab2['FLAG']=='abnormal']
date_only(lab2,'CHARTTIME')
lab2 = lab2.drop(columns=['FLAG'])
lab2['TYPE']='LAB'

lab2

,SUBJECT_ID,ITEMID,CHARTTIME,TYPE
2,9,50821,2149-11-14,LAB
6,9,50910,2149-11-14,LAB
7,9,50911,2149-11-14,LAB
8,9,50964,2149-11-14,LAB
19,9,50821,2149-11-14,LAB
...,...,...,...,...
9356341,96442,51221,2115-06-29,LAB
9356342,96442,51222,2115-06-29,LAB
9356343,96442,51244,2115-06-29,LAB
9356347,96442,51254,2115-06-29,LAB


## pre

In [5]:
pre = pd.read_csv('./data/PRESCRIPTIONS.csv')
pre2 = pre.copy()

sub_list = lab['SUBJECT_ID'].unique()
pre2 = pre2[pre2['SUBJECT_ID'].isin(sub_list)]
pre2 = pre2[['SUBJECT_ID','STARTDATE','ENDDATE','NDC']]
date_only(pre2,'STARTDATE')
date_only(pre2,'ENDDATE')

both_null = pre2[(pre2['STARTDATE'].isnull())&(pre2['ENDDATE'].isnull())].index
ndc_null = pre2[(pre2['NDC'].isnull())].index
pre2 = pre2.drop(index=both_null)
pre2 = pre2.drop(index=ndc_null)

#enddate가 null인 값은 startdate 삽입
end_null = pre2[pre2['ENDDATE'].isnull()]
end_null['ENDDATE'] = end_null['STARTDATE']

start_null = pre2[(pre2['STARTDATE'].isnull())&(pre2['ENDDATE'].notnull())]
start_null['STARTDATE'] = start_null['ENDDATE']

pre2.loc[end_null.index] = end_null
pre2.loc[start_null.index] = start_null

pre2 = pre2.sort_values(['SUBJECT_ID','STARTDATE']).reset_index(drop=True)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-5-a821ce2f9673>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  end_null['ENDDATE'] = end_null['STARTDATE']
<ipython-input-5-a821ce2f9673>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_null['STARTDATE'] = start_null['ENDDATE']


In [6]:
# from tqdm import tqdm

# f = open('./data/PRESCRIPTIONS.csv', 'r') 
# lines = f.readlines()
# sub_list = lab['SUBJECT_ID'].unique()

# for line in tqdm(lines):
#     line = line.strip().split(',')
#     sub, startdate, enddate, itemid = line[1], line[4], line[5], line[-7]
    
#     sub
#     startdate
# #     if sub in sub_list:
        
# #     break

In [7]:
from datetime import datetime, timedelta
from tqdm import tqdm

def date_range(start, end):
    dates = [(start + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((end-start).days+1)]
    return dates

pre2_list = pre2.values.tolist()
pre2_dic = {}
lst_time = []
lst_itemid = []
lst_subid = []
for idx, row in enumerate(tqdm(pre2_list)):
    sub, start, end, itemid = row[0], row[1], row[2], row[3]
    
    day_list = date_range(start, end)
    
    # index에 맞춰서 subid, date, itemid를 넣어주기 위한 dic
    # {0: ['2149-11-09'], ...,1327249: ['2181-02-11', '2181-02-12']}
    pre2_dic[idx] = day_list
    
    # pre2 DATE열에 들어갈 lst
    lst_time.extend(pre2_dic[idx][:])

    # pre2 itemid(NDC)열에 들어갈 lst
    for _ in pre2_dic[idx]:
        lst_itemid.append(itemid)
    
    # pre2 subid열에 들어갈 lst
    for _ in pre2_dic[idx]:
        lst_subid.append(sub)  

100%|██████████| 1327249/1327249 [00:16<00:00, 81610.91it/s]


In [8]:
pre2sub = pd.DataFrame(lst_subid)
pre2time = pd.DataFrame(lst_time)
pre2item = pd.DataFrame(lst_itemid)

pre2 = pd.concat([pre2sub, pre2time, pre2item], axis = 1)
pre2.columns = ['SUBJECT_ID', 'CHARTTIME', 'ITEMID']
pre2['TYPE'] = 'PRE'
pre2

,SUBJECT_ID,CHARTTIME,ITEMID,TYPE
0,9,2149-11-09,0.0,PRE
1,9,2149-11-09,85036207.0,PRE
2,9,2149-11-09,456066270.0,PRE
3,9,2149-11-09,456066270.0,PRE
4,9,2149-11-09,338001702.0,PRE
...,...,...,...,...
5650364,99985,2181-02-10,0.0,PRE
5650365,99985,2181-02-11,0.0,PRE
5650366,99985,2181-02-12,0.0,PRE
5650367,99985,2181-02-11,781305714.0,PRE


## pro

In [9]:
pro = pd.read_csv('./data/PROCEDUREEVENTS_MV.csv')

pro2 = pro.copy()
pro2 = pro2[(pro2['SUBJECT_ID'].isin(sub_list))]
pro2 = pro2[['SUBJECT_ID','STARTTIME','ENDTIME','ITEMID']]
pro2 = pro2.reset_index(drop=True)

date_only(pro2,'STARTTIME')
date_only(pro2,'ENDTIME')

# TIME 만드는 함수
from datetime import datetime, timedelta

def date_range(start, end):
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

a = date_range(pro2['STARTTIME'][0], pro2['ENDTIME'][0]) #test
b = date_range(pro2['STARTTIME'][1], pro2['ENDTIME'][1]) #test

In [10]:
from datetime import datetime, timedelta
from tqdm import tqdm

def date_range(start, end):
    dates = [(start + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((end-start).days+1)]
    return dates

pro2_list = pro2.values.tolist()
pro2_dic = {}
lst_time = []
lst_itemid = []
lst_subid = []
for idx, row in enumerate(tqdm(pro2_list)):
    sub, start, end, itemid = row[0], row[1], row[2], row[3]
    
    day_list = date_range(start, end)
    
    # index에 맞춰서 subid, date, itemid를 넣어주기 위한 dic
    # {0: ['2149-11-09'], ...,1327249: ['2181-02-11', '2181-02-12']}
    pro2_dic[idx] = day_list
    
    # pre2 DATE열에 들어갈 lst
    lst_time.extend(pro2_dic[idx][:])

    # pre2 itemid(NDC)열에 들어갈 lst
    for _ in pro2_dic[idx]:
        lst_itemid.append(itemid)
    
    # pre2 subid열에 들어갈 lst
    for _ in pro2_dic[idx]:
        lst_subid.append(sub)

pro2sub = pd.DataFrame(lst_subid)
pro2time = pd.DataFrame(lst_time)
pro2item = pd.DataFrame(lst_itemid)

100%|██████████| 79678/79678 [00:00<00:00, 155156.93it/s]


In [11]:
pro2 = pd.concat([pro2sub, pro2time, pro2item], axis = 1)
pro2.columns = ['SUBJECT_ID', 'CHARTTIME', 'ITEMID']
pro2['TYPE'] = ["PRO"]*len(pro2)
pro2

,SUBJECT_ID,CHARTTIME,ITEMID,TYPE
0,31608,2133-01-10,224385,PRO
1,31608,2133-01-10,225792,PRO
2,31608,2133-01-11,225792,PRO
3,31608,2133-01-12,225792,PRO
4,31608,2133-01-13,225792,PRO
...,...,...,...,...
164937,41958,2155-08-10,225204,PRO
164938,41958,2155-08-09,225402,PRO
164939,41958,2155-08-09,225459,PRO
164940,41958,2155-08-09,225966,PRO


## merge

In [12]:
lab2.columns
pre2.columns
pro2.columns

Index(['SUBJECT_ID', 'ITEMID', 'CHARTTIME', 'TYPE'], dtype='object')

Index(['SUBJECT_ID', 'CHARTTIME', 'ITEMID', 'TYPE'], dtype='object')

Index(['SUBJECT_ID', 'CHARTTIME', 'ITEMID', 'TYPE'], dtype='object')

In [13]:
m1 = pd.merge(lab2, pre2, on=['SUBJECT_ID', 'ITEMID', 'CHARTTIME', 'TYPE'], how='outer')
total_data = pd.merge(m1, pro2, on=['SUBJECT_ID', 'ITEMID', 'CHARTTIME', 'TYPE'], how='outer')
total_data = total_data.sort_values(['SUBJECT_ID','CHARTTIME']).reset_index(drop=True)
total_data = total_data.astype({'ITEMID':'int'})
total_data

,SUBJECT_ID,ITEMID,CHARTTIME,TYPE
0,9,50882,2149-11-09,LAB
1,9,50931,2149-11-09,LAB
2,9,50971,2149-11-09,LAB
3,9,51275,2149-11-09,LAB
4,9,51288,2149-11-09,LAB
...,...,...,...,...
9221915,99985,51288,2181-03-06,LAB
9221916,99985,51279,2182-03-14,LAB
9221917,99985,51006,2182-03-14,LAB
9221918,99985,51221,2182-03-14,LAB


In [14]:
# total_data -> csv로 저장
total_data.to_csv('total_data.csv', index=False)

## X : 7799 x 10 x 4069

In [15]:
dic_sub2idx = {}
for i, j in enumerate(total_data['SUBJECT_ID'].unique()):
    dic_sub2idx[j] = i

dic_item2idx = {}
for i, j in enumerate(total_data['ITEMID'].sort_values().unique()):
    dic_item2idx[j] = i

admission = pd.read_csv('/project/data/ADMISSIONS.csv')
admission = admission[admission['SUBJECT_ID'].isin(total_data['SUBJECT_ID'].unique())]

# DISCHTIME열 시간 제거
import datetime as dt
admission['DISCHTIME'] = pd.to_datetime(admission['DISCHTIME'])
admission['DISCHTIME'] = admission['DISCHTIME'].dt.date

dic_sub2final_date = dict(admission.groupby('SUBJECT_ID')['DISCHTIME'].max())

In [16]:
from datetime import timedelta
from tqdm import tqdm
import datetime

x = np.zeros((7799,10,4069))

IF = open('total_data.csv','r')
line = IF.readline()
for line in tqdm(IF):
    ss = line.strip('\n').split(',')
    sub, item, charttime = int(ss[0]), int(ss[1]), datetime.date.fromisoformat(ss[2])
    subidx = dic_sub2idx[sub]
    itemidx = dic_item2idx[item]
    finaldate = dic_sub2final_date[sub]
    dateidx = -(finaldate - charttime).days + 10
    
    if (dateidx < 0) or (dateidx > 9):
        continue
    else:
        x[subidx, dateidx, itemidx] = 1
        
IF.close()
print(x.sum())

9221920it [00:15, 607159.37it/s]


2042722.0


In [17]:
# x save
np.save('x_(7799,10,4069).npy',x)

# x load
arr = np.load("x_(7799,10,4069).npy")